# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
#nltk.download()
import pandas as pd
from sqlalchemy import *
from sklearn.metrics import classification_report, accuracy_score, fbeta_score, make_scorer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import multioutput
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
nltk.download('stopwords')
from sklearn.metrics import fbeta_score, make_scorer
import pickle
from sklearn.multioutput import MultiOutputClassifier


import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\59276\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\59276\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\59276\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
from sqlalchemy import *

orcl_connect_string = 'sqlite:///InsertDatabaseName.db'
engine = create_engine(orcl_connect_string)

connection = engine.connect()

if engine.has_table(table_name='DisasterResponse'):
    print ('Yes, has the table')
else:
    print ('No table')

No table


C:\Users\59276\AppData\Local\Temp/ipykernel_26588/29032173.py:8: SADeprecationWarning: The Engine.has_table() method is deprecated and will be removed in a future release.  Please refer to Inspector.has_table(). (deprecated since: 1.4)
  if engine.has_table(table_name='DisasterResponse'):


In [44]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse',con=engine)
X = df.message[:100]
Y = df.iloc[:100,4:]

In [50]:
print(X)

0     Weather update - a cold front from Cuba that c...
1               Is the Hurricane over or is it not over
2                       Looking for someone but no name
3     UN reports Leogane 80-90 destroyed. Only Hospi...
4     says: west side of Haiti, rest of the country ...
                            ...                        
95    We are about 500 people. We're in Martissant 2...
96         Digicel let me go through, it's an emergency
97                Where can I find Capital Bank please?
98    We are in Plaine du Cul-de-sac. Where's the ne...
99    I am listening to Radio Dabon. We need help an...
Name: message, Length: 100, dtype: object


In [51]:
print(Y)

                                           categories          0          1  \
0   related-1;request-0;offer-0;aid_related-0;medi...  related-1  request-0   
1   related-1;request-0;offer-0;aid_related-1;medi...  related-1  request-0   
2   related-1;request-0;offer-0;aid_related-0;medi...  related-1  request-0   
3   related-1;request-1;offer-0;aid_related-1;medi...  related-1  request-1   
4   related-1;request-0;offer-0;aid_related-0;medi...  related-1  request-0   
..                                                ...        ...        ...   
95  related-1;request-1;offer-0;aid_related-0;medi...  related-1  request-1   
96  related-0;request-0;offer-0;aid_related-0;medi...  related-0  request-0   
97  related-0;request-0;offer-0;aid_related-0;medi...  related-0  request-0   
98  related-1;request-0;offer-0;aid_related-0;medi...  related-1  request-0   
99  related-1;request-1;offer-0;aid_related-1;medi...  related-1  request-1   

          2              3               4         

### 2. Write a tokenization function to process your text data

In [32]:
def tokenize(text):
     # Normalize text: keep a-z A-Z and all number;
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
   
    
    #tokenize
    words = word_tokenize (text)
    
    #stemming
    
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    stemmed = [stemmer.stem(word) for word in words if word not in stop_words]
   
    return stemmed
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [35]:
#initialize thhe pipleline 
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', multioutput.MultiOutputClassifier (RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 1)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001FEECD2C700>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [49]:
y_pred = pipeline.predict (X_test)
for i,col in enumerate(Y):
    print(col, classification_report(y_test.iloc[:,i],y_pred[:,i]) )

categories                                                                                                                                                                                                                                                                                                                                                                                                                                                                precision    recall  f1-score   support

related-0;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0       0.50      1.00      0.67         6
related-1;requ

C:\Users\59276\anaconda333\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\59276\anaconda333\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\59276\anaconda333\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\59276\anaconda333\lib\site-packages\sklearn\metr

### 6. Improve your model
Use grid search to find better parameters. 

In [67]:
parameters = {
                'clf__estimator__n_estimators': [10, 30],
                'clf__estimator__min_samples_split': [2, 5]
              }

cv = GridSearchCV (pipeline,parameters)
cv.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001FEECD2C700>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5],
                         'clf__estimator__n_estimators': [10, 30]})

In [56]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 10}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [58]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', multioutput.MultiOutputClassifier (RandomForestClassifier(min_samples_split=2, n_estimators= 10)))
        ])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [69]:
y_train

,categories,0,1,2,3,4,5,6,7,8,...,26,27,28,29,30,31,32,33,34,35
35,related-1;request-0;offer-0;aid_related-0;medi...,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
94,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
27,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-1,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-1,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-1
46,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-1
38,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,related-1;request-0;offer-1;aid_related-0;medi...,related-1,request-0,offer-1,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
9,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-1,medical_products-1,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-1
72,related-1;request-0;offer-0;aid_related-1;medi...,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-1,other_infrastructure-0,weather_related-1,floods-0,storm-0,fire-0,earthquake-1,cold-0,other_weather-0,direct_report-0
12,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-1


In [79]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [80]:
# Try using SVM 
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])

parameters2 = {
               'clf__estimator__learning_rate': [0.1, 0.3],
               'clf__estimator__n_estimators': [100, 200]
}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2)
cv2.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001FEECD2C700>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                                                                                 max_depth=1))))]),
             param_grid={'clf__estimator__learning_rate': [0.1, 0.3],
                         'clf__estimator__n_estimators': [100, 200]})

In [82]:
# Best parameters set
cv2.best_params_


{'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 100}

In [85]:
y_pred = cv2.predict(X_train)
print(y_pred)

[['related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'
  'related-1' 'request-0' ... 'cold-0' 'other_weather-0'
  'direct_report-0']
 ['related-1;request-1;offer-0;aid_related-1;medical_help-0;medical_products-1;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'
  'related-1' 'request

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.